# Automate County GeoJSON Creation

## Part 1: Introduction

This demonstration is the Jupyter Notebook version of ***countygeojson*** script, which is used to convert county shapefile to geojson and process it like rounding coordinates to 2 decimal places and dropping unnecessary properties.

## Part 2: Preparation

We will be using **Jupyter Notebook(anaconda 3)** to edit and run the script. Information on Anaconda installation can be found <a href='https://docs.anaconda.com/anaconda/install/'>here</a>. Please note that this script is running on Python 3.

To run this script you need:
- county shapefile stored in **county** folder
- directory path (**county** folder > **state**.geojson)

The script currently prints one GeoJSON file:
- **state**.geojson

>Original created on Dec 27 2020<br>
@author: Yijing Zhou @YijingZhou33

## Part 3: Get Started

###  Step 1: Import modules

In [1]:
import os
import pandas as pd
import json
import folium
import numpy as np
import geopandas as gpd
import string

### Step 2: Manual items to change

In [9]:
##### Manually changed items #####
state = 'Iowa'

## Part 4: Build up county GeoJSON schema

### Step 1: Convert shapefile to JSON

In [10]:
## read shapefile
## note that you may need to change the file path (county folder > shapefile folder > shapefile)
county_shp = gpd.read_file(os.path.join('county', 'Iowa_County_shp', 'County.shp'))

## convert shapefile to geojson
county_shp.to_file(os.path.join('county', state+'.geojson'), driver='GeoJSON')
county_geojson = gpd.read_file(os.path.join('county', state+'.geojson'))

## convert geojson to json 
## display features properties as dataframe
county_json = json.loads(county_geojson.to_json())
df_allCounty = pd.json_normalize(county_json['features'])

df_allCounty.head()

,id,type,properties.AREA_ACRE,properties.COUNTY_FIP,properties.COUNTY_NAM,properties.COUNTY_NUM,properties.GLOBALID,properties.OBJECTID,properties.PERIMETER_,properties.REST_UPDAT,properties.REST_UTC_O,properties.SHAPE__Are,properties.SHAPE__Len,geometry.type,geometry.coordinates
0,0,Feature,398934.11931,103,JOHNSON,52,835bf24f-1075-42bf-8eea-88b713c0ece5,6986,177089.53525,2020-06-11,-05:00,2.895182e+09,237123.956584,Polygon,"[[[-91.4815921886641, 41.8597585301547, 0.0], ..."
1,1,Feature,283811.45010,9,AUDUBON,5,ba36079f-6d0d-4a55-b167-9c71642d3036,6987,144871.44205,2020-06-11,-05:00,2.060552e+09,194066.658266,Polygon,"[[[-95.092861863321, 41.8493676039035, 0.0], [..."
2,2,Feature,448301.90406,85,HARRISON,43,9dd25e42-273d-4027-8693-a7b6e683803d,6988,198356.32574,2020-06-11,-05:00,3.254645e+09,265573.694964,Polygon,"[[[-96.1092712090134, 41.8359581300276, 0.0], ..."
3,3,Feature,492080.89689,167,SIOUX,84,ee9dd10d-71cf-43fd-97cc-c6534fbfd005,6989,221035.11225,2020-06-11,-05:00,3.735044e+09,302578.333150,Polygon,"[[[-96.5490372414588, 43.2601308133559, 0.0], ..."
4,4,Feature,376221.12421,119,LYON,60,b6607aca-f4ea-41f3-91f0-d47e63c27901,6990,194503.81366,2020-06-11,-05:00,2.883183e+09,267448.608239,Polygon,"[[[-96.5861228072531, 43.490771142258, 0.0], [..."


### Step 2: Clean up unnecessary columns

In [11]:
## drop and rename columns
df_allCounty = df_allCounty[['properties.COUNTY_NAM', 'geometry.coordinates']].rename(
    columns={'properties.COUNTY_NAM':'County', 'geometry.coordinates':'boundingBox'})

## convert strings to titlecase
df_allCounty['County'] = df_allCounty['County'].apply(lambda row: string.capwords(row))

## Caution! 
## you may need to manually check if any county name has abbreviation, special characters and so on...
## df_allCounty['County'] = df_allCounty['County'].replace(['Obrien'],"O'Brien")

df_allCounty.head()

,County,boundingBox
0,Johnson,"[[[-91.4815921886641, 41.8597585301547, 0.0], ..."
1,Audubon,"[[[-95.092861863321, 41.8493676039035, 0.0], [..."
2,Harrison,"[[[-96.1092712090134, 41.8359581300276, 0.0], ..."
3,Sioux,"[[[-96.5490372414588, 43.2601308133559, 0.0], ..."
4,Lyon,"[[[-96.5861228072531, 43.490771142258, 0.0], [..."


### Step 3: Round coordinates to 2 decimal places

In [12]:
def round_coordinates(l, precision):
    def round_element(e):
        if isinstance(e, list):
            return round_coordinates(e, precision)
        else:
            return round(e, precision)
    return [round_element(e) for e in l]

df_allCounty['boundingBox'] = round_coordinates(df_allCounty['boundingBox'], 2)
df_allCounty.head()

,County,boundingBox
0,Johnson,"[[[-91.48, 41.86, 0.0], [-91.48, 41.86, 0.0], ..."
1,Audubon,"[[[-95.09, 41.85, 0.0], [-95.09, 41.86, 0.0], ..."
2,Harrison,"[[[-96.11, 41.84, 0.0], [-96.11, 41.84, 0.0], ..."
3,Sioux,"[[[-96.55, 43.26, 0.0], [-96.53, 43.26, 0.0], ..."
4,Lyon,"[[[-96.59, 43.49, 0.0], [-96.59, 43.49, 0.0], ..."


## Part 5: Create County GeoJSON

### Step 1: Create geojson features

In [14]:
# create_geojson_features 
def create_geojson_features(df):
    print('> Creating GeoJSON features...')
    features = []
    geojson = {
        'type': 'FeatureCollection',
        'features': features
    }
        
    for _, row in df.iterrows():
        if type(row['boundingBox'][0][0][0]) is float:
            geometry_type = 'Polygon'
        else:
            geometry_type = 'MultiPolygon'
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':geometry_type, 
                'coordinates':row['boundingBox']
            },
            'properties': {
                'county': row['County']
            }
        }

        features.append(feature)
    return geojson

allCounty_geojson = create_geojson_features(df_allCounty)

> Creating GeoJSON features...


### Step 2: Generate geojson file

In [15]:
with open(os.path.join('county', state+'.geojson'), 'w') as txtfile:
    json.dump(allCounty_geojson, txtfile)
print('> Creating GeoJSON file...')

> Creating GeoJSON file...


## Part 6: Draw the index maps

In [16]:
print('> Making map...')
## change the location here to zoom to the center
m = folium.Map(location = [42.3756, -93.6397], control_scale = True, zoom_start = 7)

## check if the indexmap geojson files can be rendered properly
folium.GeoJson(open(os.path.join('county', state+'.geojson'), 'r').read(),
               show = True).add_to(m)
m

> Making map...
